In [153]:
import numpy as np
import pandas as pd
import sklearn
import sklearn.ensemble
import lime
import lime.lime_tabular

# util
from xaibenchmark import load_adult as la

In [154]:
data = la.load_csv_data('adult', root_path='../data')

def preprocess(*data_df): 
    def process_single(df):
        
        cat_df = pd.get_dummies(df, columns=data.categorical_features.keys())
        missing_cols = {cat+'_'+str(attr) for cat in data.categorical_features \
                        for attr in data.categorical_features[cat]} - set(cat_df.columns)
        for c in missing_cols:
            cat_df[c] = 0
            
        cont_idx = list(set(data.data.keys()) - set(data.categorical_features.keys()))
        cat_idx = [cat+'_'+str(attr) for cat in data.categorical_features \
                   for attr in data.categorical_features[cat]]
        idx = cont_idx + cat_idx
        return cat_df[idx]
        
    # Preprocess function for one-hot encoding categorical data
    return [process_single(df) for df in data_df]

train, dev, test = preprocess(data.data, data.data_dev, data.data_test)
labels_train, labels_dev, labels_test = data.target, data.target_dev, data.target_test

In [3]:
rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train, labels_train.to_numpy().reshape(-1))

RandomForestClassifier(n_estimators=500)

In [4]:
y_pred = rf.predict(dev)

In [5]:
y_true = labels_dev.to_numpy().reshape(-1)

In [6]:
print('Classification report')
print('{:->60}'.format(''))
print(sklearn.metrics.classification_report(y_true, y_pred))

Classification report
------------------------------------------------------------
              precision    recall  f1-score   support

       <=50K       0.88      0.93      0.90      7436
        >50K       0.72      0.61      0.66      2332

    accuracy                           0.85      9768
   macro avg       0.80      0.77      0.78      9768
weighted avg       0.85      0.85      0.85      9768



In [7]:
import xaibenchmark as xb
import scipy as sp

In [192]:
class LimeExplainer(xb.Explainer):
    
    def __init__(self, train_data, train_labels, predict_fn, feature_names, target_names, categorical_features=None, discretize_continuous=True):
        
        self.explainer = lime.lime_tabular.LimeTabularExplainer(train_data, feature_names=feature_names,
                                                   class_names=target_names, categorical_features=None,
                                                   discretize_continuous=discretize_continuous)
        self.train = train_data
        self.train_labels = train_labels
        self.predict_fn = predict_fn
        self.kernel_width = np.sqrt(train_data.shape[1]) * .75

    def explain_instance(self, instance, predictor, num_features=10):
        self.explanation = self.explainer.explain_instance(instance, predictor, num_features=num_features)
        self.instance = instance
        self.weighted_instances = self.get_weighted_instances()

        return self.explanation
    
    @xb.metric
    def area(self):
        """
        Area that is covered by the kernel in high dimension of the feature space.
        """
        kernel_width = np.sqrt(self.train.shape[1]) * .75
        kernel_dimension = self.train.shape[1]
        return (kernel_width * np.sqrt(2*np.pi))**kernel_dimension

    @xb.metric
    def coverage(self):
        """
        Proportion of instances covered in the area
        """
        weighted_instances = self.weighted_instances
        return sum([weight for _, weight in self.weighted_instances]) / len(self.weighted_instances)
    
    @xb.metric
    def furthest_distance(self):
        kernel_width = np.sqrt(self.train.shape[1]) * .75
        def kernel(distance):
            return np.sqrt(np.exp(-distance**2/kernel_width**2))
        training_instances = self.train.to_numpy()
        distance_instances = (self.distance(self.instance, instance) for instance in training_instances)
        weighted_distances = (distance * kernel(distance) for distance in distance_instances)
        return sum(weighted_distances)

    @xb.metric
    def accuracy(self):
        """
        Proportion of instances in the explanation neighborhood that shares the same output label by the
        explainer and the ML model
        """

        ml_preds = self.predict_fn.predict_proba(self.train)[:, 1]
        ml_preds = ml_preds > 0.5
        exp_preds = [self.predict_instance_surrogate(instance) for instance, _ in self.weighted_instances]
        exp_preds = np.array(exp_preds) > 0.5
        return  (ml_preds == exp_preds).sum() / len(exp_preds)

    @xb.metric
    def balance(self):
        """
        Proportion of instances in the explanation neighborhood that has been assigned label 1 by the
        explanation model
        """
        exp_preds = [self.predict_instance_surrogate(instance) for instance, _ in self.weighted_instances]
        exp_preds = np.array(exp_preds) > 0.5
        return exp_preds.sum() / len(exp_preds)


    @xb.utility
    def distance(self, x, y):
        return np.linalg.norm(x-y)
    
    @xb.utility
    def get_weighted_instances(self):     
        if hasattr(self, 'explanation'):
            kernel_width = np.sqrt(self.train.shape[1]) * .75
            def kernel(distance):
                return np.sqrt(np.exp(-distance**2/kernel_width**2))
            return [(instance, kernel(self.distance(self.instance, instance))) \
                    for instance in self.train.to_numpy()]
        return []
    
    @xb.utility
    def get_explained_instance(self):
        return self.instance

    @xb.utility
    def get_training_data(self):
        return self.train

    @xb.utility
    def predict_instance_surrogate(self, instance):
        return np.clip(self.explanation.intercept[1] + sum(weight * \
               ((instance - self.explainer.scaler.mean_)/self.explainer.scaler.scale_ )[idx] \
               for idx, weight in self.explanation.local_exp[1]), 0, 1)

In [193]:
exp = LimeExplainer(train, labels_train, rf, train.keys(), data.target_names, discretize_continuous=False)

In [194]:
exp.explain_instance(test.iloc[0], rf.predict_proba, num_features=10)

In [195]:
exp.report()

{('accuracy', 0.8148554380730926),
 ('area', 2.589477453233665e+139),
 ('balance', 0.06440573860395736),
 ('coverage', 1.221577124110411e-10),
 ('furthest_distance', 0.00010993021234123974)}

In [149]:
exp.infer_metrics()

inferred metrics: {'area', 'furthest_distance', 'balance', 'inverse_coverage', 'accuracy', 'coverage'}


In [150]:
exp.report()

{('accuracy', nan),
 ('area', 2.589477453233665e+139),
 ('balance', nan),
 ('coverage', 2.8498848684316357e-07),
 ('furthest_distance', 0.16073684608677166),
 ('inverse_coverage', 3508913.6795562045)}

In [151]:
exp.explain_instance(test.iloc[10], rf.predict_proba, num_features=10)
exp.report()

/Users/juliefang/PycharmProjects/lab21-XAI-benchmark/xaibenchmark/explainer.py:60: RuntimeWarning: divide by zero encountered in double_scalars
  ({'coverage'}, 'inverse_coverage', metric(lambda : 1 / self.coverage())),


{('accuracy', nan),
 ('area', 2.589477453233665e+139),
 ('balance', nan),
 ('coverage', 0.0),
 ('furthest_distance', 0.0),
 ('inverse_coverage', inf)}

In [152]:
exp.infer_metrics()
exp.report()

inferred metrics: {'area', 'furthest_distance', 'balance', 'inverse_coverage', 'accuracy', 'coverage'}


/Users/juliefang/PycharmProjects/lab21-XAI-benchmark/xaibenchmark/explainer.py:60: RuntimeWarning: divide by zero encountered in double_scalars
  ({'coverage'}, 'inverse_coverage', metric(lambda : 1 / self.coverage())),


{('accuracy', nan),
 ('area', 2.589477453233665e+139),
 ('balance', nan),
 ('coverage', 0.0),
 ('furthest_distance', 0.0),
 ('inverse_coverage', inf)}

In [ ]:
[i for i, w in exp.weighted_instances][0].shape

In [82]:
exp.train.iloc[0].shape

(108,)